In [ ]:
import os
import json
import anthropic
from dotenv import load_dotenv
from openai import Client
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

load_dotenv()

SCOPES = ["https://www.googleapis.com/auth/calendar"]


if os.path.exists("./res/token.json"):
    creds = Credentials.from_authorized_user_file("./res/token.json", SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file("./res/credentials.json", SCOPES)
    creds = flow.run_local_server(port=0)
    with open("./res/token.json", "w") as token:
        token.write(creds.to_json())

service = build("calendar", "v3", credentials=creds)

client = Client()

### Multi-Turn Chat
- User-Assistant 간의 대화 히스토리를 저장 후 LLM API에 입력
  - 이전 대화 내용을 기억하고 기존 대화 맥락을 참고해서 알맞는 답변 생성
- 턴 수가 많아질수록 LLM API 답변 퀄리티가 낮아지거나 일부 내용이 있더라도 찾아내지 못 할 가능성 존재
- 입력 토큰 수 증가

In [6]:
messages = []

In [ ]:
PROMPT = "2026년 2월 16일 12시부터 1시간 짜리 점심 일정 생성해주세요."


# 1. 함수 생성
def create_event(summary, start, end):
    event = {
        "summary": summary,
        "start": {
            "dateTime": start,
            "timeZone": "Asia/Seoul",
        },
        "end": {
            "dateTime": end,
            "timeZone": "Asia/Seoul",
        },
    }
    event = service.events().insert(calendarId="primary", body=event).execute()
    print("Event created: %s" % (event.get("htmlLink")))
    return event


# 2. tools 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "create_event",
            "description": "Create a new Calendar Event",
            "parameters": {
                "type": "object",
                "required": ["summary", "start", "end"],
                "properties": {
                    "summary": {
                        "type": "string",
                        "description": "Name of Google Calender Event",
                    },
                    "start": {
                        "type": "string",
                        "description": "Starting date of Google Calender Event in UTC+9 i.e. 2026-02-05T09:00:00+09:00",
                    },
                    "end": {
                        "type": "string",
                        "description": "Ending date of Google Calender Event in UTC+9 i.e. 2026-02-05T10:00:00+09:00",
                    },
                },
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

# 3. LLM 1차 호출: 도구 호출 여부 결정
response = client.chat.completions.create(
    model="gpt-4o-mini",
    max_tokens=1024,
    tools=tools,
    messages=[{"role": "user", "content": PROMPT}],
)

messages.append({"role": "user", "content": PROMPT})


# AI가 생성한 메시지 객체 저장
assistant_message = response.choices[0].message

# 4. 도구 호출 실행
if assistant_message.tool_calls:
    tool_call = assistant_message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)

    # 실제 함수 실행
    event_result = create_event(
        summary=args["summary"], start=args["start"], end=args["end"]
    )

    # 5. LLM API 최종 호출: 실행 결과를 토대로 답변 생성
    final_response = client.chat.completions.create(
        model="gpt-4o-mini",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": PROMPT},
            assistant_message,  # AI가 보냈던 도구 호출 요청 메시지
            {
                "role": "tool",
                "tool_call_id": tool_call.id,  # 이전 단계의 ID와 반드시 일치해야 함
                "content": f"Event created: {event_result.get('htmlLink')}",
            },
        ],
    )

messages.append(
    {"role": "assistant", "content": final_response.choices[0].message.content}
)

Event created: https://www.google.com/calendar/event?eid=MjF1cXJwdTAycG5yb2R0Y2RiZTI1bWYzcm8gcWFzZDIwMjE5QG0


In [8]:
messages

[{'role': 'user', 'content': '2026년 2월 16일 12시부터 1시간 짜리 점심 일정 생성해주세요.'},
 {'role': 'assistant',
  'content': '2026년 2월 16일 12시부터 1시간 짜리 점심 일정이 생성되었습니다. [여기서 확인할 수 있습니다.](https://www.google.com/calendar/event?eid=MjF1cXJwdTAycG5yb2R0Y2RiZTI1bWYzMycm8gcWFzZDIwMjE5QG0)'}]

In [9]:
print(final_response.choices[0].message.content)

2026년 2월 16일 12시부터 1시간 짜리 점심 일정이 생성되었습니다. [여기서 확인할 수 있습니다.](https://www.google.com/calendar/event?eid=MjF1cXJwdTAycG5yb2R0Y2RiZTI1bWYzMycm8gcWFzZDIwMjE5QG0)


In [ ]:
new_message = "제가 이전에 어떤 요청을 드렸나요?"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    max_tokens=1024,
    tools=tools,
    messages=[{"role": "user", "content": new_message}],
)
print(response.choices[0].message.content)

죄송하지만, 이전 대화 내용을 조회할 수 없습니다. 하지만 현재 도움이 필요하신 내용이나 질문이 있으시면 말씀해 주세요!


In [ ]:
new_message = "저의 가장 최근 요청은 어떤 내용이었나요?"

messages.append({"role": "user", "content": new_message})

response = client.chat.completions.create(
    model="gpt-4o-mini",
    max_tokens=1024,
    tools=tools,
    messages=messages,
)
print(response.choices[0].message.content)

귀하의 가장 최근 요청은 2026년 2월 16일 12시부터 1시간 동안의 점심 일정을 생성하는 것이었습니다.
